In [1]:
import os
import re

from rdkit import Chem
from openbabel import pybel
import parameter_extractor as pe
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem.Draw.MolDrawing import DrawingOptions

IPythonConsole.molSize = (300, 300)  # Change image size
DrawingOptions.bondLineWidth = 1.8

# Gaussian Input Files Generation
This notebook generates `.gjf` and script files to submit the calculations to gaussian.

In [21]:
directory = "./out/pending/"

In [32]:
for file in os.listdir(directory):
    if not file.endswith(".out"):
        continue
    path = directory + file
    ligand_name = file.split(".")[0]
    submit_filename = f"{ligand_name}_SPE"
    submit_file = f"""#!/bin/csh
#$ -N {submit_filename}
#$ -pe smp 14
#$ -q iqtc08.q
#$ -S /bin/csh
#$ -cwd
#$ -o $HOME/javier/{submit_filename}.log
#$ -e $HOME/javier/{submit_filename}.err
#$ -m e
#$ -M javier.eusamio@ub.edu

# Load the modules needed
source /etc/profile.d/modules.csh
module load gaussian/g16b01

cd $TMPDIR
cp -pr $HOME/javier/{submit_filename}.gjf .

# Run the job
g16 < {submit_filename}.gjf > {submit_filename}.out

# Copy the results to our home directory
cp {submit_filename}.out $HOME/javier/
cp {submit_filename}.chk $HOME/javier/"""
    out = next(pybel.readfile("out", path))
    gjf = out.write("gjf")
    gjf = re.sub(
        r"!Put Keywords Here, check Charge and Multiplicity.\n#\n\n",
        fr"%chk={ligand_name}_SPE\n%nprocshared=14\n%mem=8GB\n# nmr=giao b3lyp/genecp pop=nbo scfcyc=200\n\n",
        gjf
    )
    gjf = re.sub(" " + path, ligand_name + "_SPE", gjf)
    comp = pe.ParameterExtractor(path, include_spe_prop=False)
    symbols_list = {a.GetSymbol() for a in comp.mol.GetAtoms() if a.GetAtomicNum() <= 36}
    basis_set_ecp = f"""{" ".join(symbols_list)}  0
TZVP
****
Rh  0
S    2   1.00
     17.000000000           -0.16690803139
     13.910581694            0.34235001652
S    1   1.00
      5.2481265288           1.0000000
S    1   1.00
      1.2262575928           1.0000000
S    1   1.00
      0.53930216349          1.0000000
S    1   1.00
      0.10130730377          1.0000000
S    1   1.00
      0.37124139005D-01      1.0000000
P    4   1.00
     11.767103631            0.59494859388D-01
      6.7485133083          -0.23735853477
      1.7502679834           0.49019334303
      0.84321166133          0.50623933751
P    1   1.00
      0.38295544759          1.0000000
P    1   1.00
      0.11500000000          1.0000000
P    1   1.00
      0.37000000000D-01      1.0000000
D    4   1.00
     19.857830136            0.66960778187D-02
     10.061378139           -0.21981738213D-01
      2.2619546477           0.37918706236
      0.97098845035          0.67289976592
D    1   1.00
      0.38391195297          1.0000000
D    1   1.00
      0.13537026904          1.0000000
F    1   1.00
      1.0949900              1.0000000
****

Rh 0
RH-ECP     3     28
f potential
  2
2     12.3100000            -30.09345572
2      6.1600000             -5.21848192
s-f potential
  4
2     11.7200000            225.34775353
2      5.8200000             32.82318898
2     12.3100000             30.09345572
2      6.1600000              5.21848192
p-f potential
  4
2     10.4200000            158.70941159
2      5.4500000             26.44410049
2     12.3100000             30.09345572
2      6.1600000              5.21848192
d-f potential
  4
2      8.8200000             62.75862572
2      3.8700000             10.97871947
2     12.3100000             30.09345572
2      6.1600000              5.21848192

"""
    final_path = "./gjf/spe/" + submit_filename
    if not os.path.exists(final_path):
        os.makedirs(final_path)

    with open(f"{final_path}/{ligand_name}_SPE.gjf", "w") as f:
        f.write(gjf + basis_set_ecp)
    with open(f"{final_path}/{submit_filename}", "w", newline='\n') as f:
        f.write(submit_file)
    print(f"Ligand {ligand_name} done!")

Ligand dcpa done!
Ligand dcPAMP done!
Ligand diprpa done!
Ligand DS-1 done!
Ligand DS-10 done!
Ligand DS-11 done!
Ligand DS-12 done!
Ligand DS-13 done!
Ligand DS-14 done!
Ligand DS-15 done!
Ligand DS-16 done!
Ligand DS-17 done!
Ligand DS-18 done!
Ligand DS-19 done!
Ligand DS-2 done!
Ligand DS-20 done!
Ligand DS-21 done!
Ligand DS-22 done!
Ligand DS-23 done!
Ligand DS-24 done!
Ligand DS-25 done!
Ligand DS-26 done!
Ligand DS-27 done!
Ligand DS-28 done!
Ligand DS-29 done!
Ligand DS-3 done!
Ligand DS-30 done!
Ligand DS-31 done!
Ligand DS-32 done!
Ligand DS-33 done!
Ligand DS-34 done!
Ligand DS-35 done!
Ligand DS-36 done!
Ligand DS-37 done!
Ligand DS-38 done!
Ligand DS-39 done!
Ligand DS-4 done!
Ligand DS-40 done!
Ligand DS-41 done!
Ligand DS-42 done!
Ligand DS-43 done!
Ligand DS-44 done!
Ligand DS-45 done!
Ligand DS-46 done!
Ligand DS-48 done!
Ligand DS-49 done!
Ligand DS-5 done!
Ligand DS-50 done!
Ligand DS-52 done!
Ligand DS-53 done!
Ligand DS-54 done!
Ligand DS-55 done!
Ligand DS-56 don

This cell creates input files for `SPE_NoRh` structures.

In [2]:
directory = "./out/pending/"

In [34]:
for file in os.listdir(directory):
    if not file.endswith(".out"):
        continue
    path = directory + file
    ligand_name = file.split(".")[0]
    submit_filename = f"{ligand_name}_SPE_NoRh"
    submit_file = f"""#!/bin/csh
#$ -N {submit_filename}
#$ -pe smp 14
#$ -q iqtc08.q
#$ -S /bin/csh
#$ -cwd
#$ -o $HOME/javier/{submit_filename}.log
#$ -e $HOME/javier/{submit_filename}.err
#$ -m e  
#$ -M javier.eusamio@ub.edu 

# Load the modules needed
source /etc/profile.d/modules.csh
module load gaussian/g16b01

cd $TMPDIR
cp -pr $HOME/javier/{submit_filename}.gjf .

# Run the job
g16 < {submit_filename}.gjf > {submit_filename}.out

# Copy the results to our home directory
cp {submit_filename}.out $HOME/javier/"""
    
    # Removes COD + Rh from molecule, converts to .gjf and adds job keywords
    comp = pe.ParameterExtractor(path, include_spe_prop=False)
    editable_mol = Chem.RWMol(comp.mol)
    cod_idx = [a.GetIdx() for a in comp.cod]
    remove_atoms = [a for a in editable_mol.GetAtoms() if a.GetIdx() in cod_idx]
    remove_atoms.append(comp.rh)
    remove_atoms = sorted(remove_atoms, key=lambda x: x.GetIdx(), reverse=True)
    for atom in remove_atoms:
        atom.SetAtomMapNum(atom.GetIdx())
        editable_mol.RemoveAtom(atom.GetIdx())
    pdb_block = Chem.MolToPDBBlock(editable_mol)
    pdb = pybel.readstring("pdb", pdb_block)
    gjf = pdb.write("gjf")
    gjf = re.sub(
        r"!Put Keywords Here, check Charge and Multiplicity.\n#\n\n",
        fr"%nprocshared=14\n%mem=8GB\n# nmr=giao b3lyp/def2TZVP pop=nbo scfcyc=200\n\n{ligand_name} SPE NoRh",
        gjf
    )

    final_path = "./gjf/NoRh/" + submit_filename
    if not os.path.exists(final_path):
        os.makedirs(final_path)
    with open(f"{final_path}/{ligand_name}_SPE_NoRh.gjf", "w") as f:
        f.write(gjf)
    with open(f"{final_path}/{submit_filename}", "w", newline='\n') as f:
        f.write(submit_file)
    print(f"Ligand {ligand_name} done!")

Ligand dcpa done!
Ligand dcPAMP done!
Ligand diprpa done!
Ligand DS-1 done!
Ligand DS-10 done!
Ligand DS-11 done!
Ligand DS-12 done!
Ligand DS-13 done!
Ligand DS-14 done!
Ligand DS-15 done!
Ligand DS-16 done!
Ligand DS-17 done!
Ligand DS-18 done!
Ligand DS-19 done!
Ligand DS-2 done!
Ligand DS-20 done!
Ligand DS-21 done!
Ligand DS-22 done!
Ligand DS-23 done!
Ligand DS-24 done!
Ligand DS-25 done!
Ligand DS-26 done!
Ligand DS-27 done!
Ligand DS-28 done!
Ligand DS-29 done!
Ligand DS-3 done!
Ligand DS-30 done!
Ligand DS-31 done!
Ligand DS-32 done!
Ligand DS-33 done!
Ligand DS-34 done!
Ligand DS-35 done!
Ligand DS-36 done!
Ligand DS-37 done!
Ligand DS-38 done!
Ligand DS-39 done!
Ligand DS-4 done!
Ligand DS-40 done!
Ligand DS-41 done!
Ligand DS-42 done!
Ligand DS-43 done!
Ligand DS-44 done!
Ligand DS-45 done!
Ligand DS-46 done!
Ligand DS-48 done!
Ligand DS-49 done!
Ligand DS-5 done!
Ligand DS-50 done!
Ligand DS-52 done!
Ligand DS-53 done!
Ligand DS-54 done!
Ligand DS-55 done!
Ligand DS-56 don

In [28]:
path = "./out/pending/diprpa.out"
ligand_name = "diprpa"

out = next(pybel.readfile("out", path))
gjf = out.write("gjf")
gjf = re.sub(
    r"!Put Keywords Here, check Charge and Multiplicity.\n#\n\n",
    fr"%chk={ligand_name}_SPE\n%nprocshared=14\n%mem=8GB\n# nmr=giao b3lyp/genecp pop=nbo scfcyc=200\n\n",
    gjf
)
gjf = re.sub(" " + path, ligand_name + "_SPE", gjf)
comp = pe.ParameterExtractor(path, include_spe_prop=False)
symbols_list = {a.GetSymbol() for a in comp.mol.GetAtoms() if a.GetAtomicNum() <= 36}
basis_set_ecp = f"""{" ".join(symbols_list)}  0
TZVP
****
Rh  0
S    2   1.00
     17.000000000           -0.16690803139
     13.910581694            0.34235001652
S    1   1.00
      5.2481265288           1.0000000
S    1   1.00
      1.2262575928           1.0000000
S    1   1.00
      0.53930216349          1.0000000
S    1   1.00
      0.10130730377          1.0000000
S    1   1.00
      0.37124139005D-01      1.0000000
P    4   1.00
     11.767103631            0.59494859388D-01
      6.7485133083          -0.23735853477
      1.7502679834           0.49019334303
      0.84321166133          0.50623933751
P    1   1.00
      0.38295544759          1.0000000
P    1   1.00
      0.11500000000          1.0000000
P    1   1.00
      0.37000000000D-01      1.0000000
D    4   1.00
     19.857830136            0.66960778187D-02
     10.061378139           -0.21981738213D-01
      2.2619546477           0.37918706236
      0.97098845035          0.67289976592
D    1   1.00
      0.38391195297          1.0000000
D    1   1.00
      0.13537026904          1.0000000
F    1   1.00
      1.0949900              1.0000000
****

Rh 0
RH-ECP     3     28
f potential
  2
2     12.3100000            -30.09345572
2      6.1600000             -5.21848192
s-f potential
  4
2     11.7200000            225.34775353
2      5.8200000             32.82318898
2     12.3100000             30.09345572
2      6.1600000              5.21848192
p-f potential
  4
2     10.4200000            158.70941159
2      5.4500000             26.44410049
2     12.3100000             30.09345572
2      6.1600000              5.21848192
d-f potential
  4
2      8.8200000             62.75862572
2      3.8700000             10.97871947
2     12.3100000             30.09345572
2      6.1600000              5.21848192

"""

with open(ligand_name + "_spe.gjf", "w") as f:
    f.write(gjf + basis_set_ecp)